In [ ]:
import random

import numpy as np
import matplotlib.pyplot as plt

import shapely.geometry as geo
import shapely.affinity as aff
from descartes.patch import PolygonPatch

In [ ]:
colours = plt.cm.viridis(np.linspace(0,1,1000))
golden_ratio = (1 + np.sqrt(5)) / 2

In [ ]:
def create_logo():

    def snake_wave_packet(x):
        x = np.array(x)
        return desired_amplitude * np.exp(-x**2/2) * np.sin(frequency_param * np.pi * x) 

    expected_y_bounds = np.ravel([-height/2, height/2])


    epsilon = 0.1
    x_init = bound - head_buffer * head_stretch

    y_vals = snake_wave_packet([x_init, x_init + epsilon])
    ydiff = np.diff(y_vals)
    y_init = y_vals[0]

    head_shift = head_buffer * head_stretch
    hypot = np.sqrt(epsilon**2 + ydiff**2)
    num_shifts = head_shift / hypot

    head_angle = np.arctan(ydiff / epsilon) / np.pi * 180
    head_middle_x = x_init + epsilon * num_shifts * neck_shring
    head_middle_y = y_init + ydiff * num_shifts * neck_shring

    tongue_init_x = head_middle_x + epsilon * num_shifts
    tongue_init_y = head_middle_y + ydiff * num_shifts

    tongue_shift_num = tongue_length / hypot

    tongue_shift_x = epsilon * tongue_shift_num
    tongue_shift_y = ydiff * tongue_shift_num

    tongue_fork_x = tongue_init_x + tongue_shift_x
    tongue_fork_y = tongue_init_y + tongue_shift_y

    tongue_base = geo.LineString(
        [(tongue_init_x, tongue_init_y), (tongue_fork_x, tongue_fork_y)]
    ).buffer(tongue_buffer)

    tongue_base

    tongue_top_fork = aff.rotate(tongue_base, fork_angles, origin=(tongue_init_x, tongue_init_y))
    tongue_top_fork = aff.translate(tongue_top_fork, tongue_shift_x, tongue_shift_y)

    tongue_bot_fork = aff.rotate(tongue_base, -fork_angles, origin=(tongue_init_x, tongue_init_y))
    tongue_bot_fork = aff.translate(tongue_bot_fork, tongue_shift_x, tongue_shift_y)

    tongue = tongue_base.union(tongue_top_fork).union(tongue_bot_fork)
    
    point = geo.Point(head_middle_x, head_middle_y)
    head = point.buffer(head_buffer)
    head = aff.scale(head, head_stretch, 1, origin=point)
    head = aff.rotate(head, head_angle, origin=point)
    head = head.union(tongue)
    
    x = np.linspace(-bound - tail_stretch, x_init, 1000)
    y = snake_wave_packet(x)
    centre_line = geo.LineString(zip(x, y))

    body = centre_line.buffer(body_buffer)
    snake = body.union(head)
    xmin, ymin, xmax, ymax = snake.bounds
    
    ring_outside = bound - bound_diff
    ring_inside = ring_outside - ring_thickness

    ring = geo.Point(0,0).buffer(ring_outside).difference(geo.Point(0,0).buffer(ring_inside))
    
    
    fig = plt.figure(1, figsize=(10,10), dpi=90)
    fig.set_frameon(True)
    ax = fig.add_subplot(111)


    patch_ring = PolygonPatch(ring, facecolor=ring_colour)
    ax.add_patch(patch_ring)

    patch_snake = PolygonPatch(snake, facecolor=snake_colour, linewidth=golden_ratio)
    ax.add_patch(patch_snake)

    plt.xlim(-bound-1, bound+1)
    plt.ylim(-bound-1, bound+1)
    
    plt.axis('off')
    plt.gca().set_position([0, 0, 1, 1])
    plt.axis('tight')
    plt.axis('equal')
    plt.savefig("logo.svg", transparent=True)

    plt.show()

In [ ]:
# Body shape
bound = 4
body_buffer = 0.21
head_buffer = body_buffer * golden_ratio
head_stretch = 1.3
frequency_param = golden_ratio
tail_stretch = 0.4
neck_shring = 0.9

expected_x_bounds = np.array([-bound - body_buffer, bound + head_buffer])
length = np.diff(expected_x_bounds)
height = length / golden_ratio
desired_amplitude = height / 2 - body_buffer + 0.3

# Tongue shape
tongue_length = 0.1
tongue_buffer = 0.035
fork_angles = 35

# Ring size
ring_thickness = golden_ratio
bound_diff = 0.15

# Colours
snake_colour = "#FFD43B"
ring_colour = "#4B8BBE" # 400


create_logo()